# Leverage Power of Machine Learning with ONNX

How to install .NET in Jupyter https://github.com/dotnet/interactive

## Instal Nuget Packages

In [ ]:
#r "nuget:Microsoft.ML,1.6.0"
#r "nuget:Microsoft.Data.Analysis,0.18.0"
#r "nuget:XPlot.Plotly, 4.0.3"
#r "nuget:Microsoft.ML.OnnxTransformer,1.6.0"
#r "nuget:Microsoft.ML.OnnxConverter,0.18.0"


In [ ]:
using System.IO;
using Microsoft.Data.Analysis;
using XPlot.Plotly;

In [ ]:
using Microsoft.AspNetCore.Html;
using Microsoft.DotNet.Interactive.Formatting;
using static Microsoft.DotNet.Interactive.Formatting.PocketViewTags;


## Load Data

In [ ]:
var data = DataFrame.LoadCsv("./SalaryData.csv");

In [ ]:
data

In [ ]:
data.Description()

In [ ]:
static T[] Shuffle<T>(T[] array)
{
    Random rand = new Random();
    for (int i = 0; i < array.Length; i++)
    {
        int r = i + rand.Next(array.Length - i);
        T temp = array[r];
        array[r] = array[i];
        array[i] = temp;
    }
    return array;
}

int[] randomIndices = Shuffle(Enumerable.Range(0, (int)data.Rows.Count).ToArray());
int testSize = (int)(data.Rows.Count * .1);
int[] trainRows = randomIndices[testSize..];
int[] testRows = randomIndices[..testSize];

DataFrame data_train = data[trainRows];
DataFrame data_test = data[testRows];

display(data_train.Rows.Count);
display(data_test.Rows.Count);

## Use ML.NET
To use ML.NET, need to create instance of MLContext.

In [ ]:
using Microsoft.ML;

In [ ]:
var context = new MLContext();

## Create Pipeline

In [ ]:
#pragma warning disable CS1701
var pipeline = context.Transforms.Concatenate("Features", "yearsExperience")
                .Append(context.Regression.Trainers.Sdca(labelColumnName: "salary"));

## Training

In [ ]:
ITransformer model = pipeline.Fit(data_train);

## Evaluate

In [ ]:
var testResults = model.Transform(data_test);

var metrics = context.Regression.Evaluate(testResults, labelColumnName: "salary", scoreColumnName: "Score");

In [ ]:
Console.WriteLine($"*************************************************");
Console.WriteLine($"*       Metrics for regression model      ");
Console.WriteLine($"*------------------------------------------------");
Console.WriteLine($"*       LossFn:        {metrics.LossFunction:0.##}");
Console.WriteLine($"*       R2 Score:      {metrics.RSquared:0.##}");
Console.WriteLine($"*       Absolute loss: {metrics.MeanAbsoluteError:#.##}");
Console.WriteLine($"*       Squared loss:  {metrics.MeanSquaredError:#.##}");
Console.WriteLine($"*       RMS loss:      {metrics.RootMeanSquaredError:#.##}");
Console.WriteLine($"*************************************************");

In [ ]:
private static readonly string MODEL_NAME = "model.onnx";

In [ ]:
data

## Save ONNX Model

In [ ]:
using System.IO;

using (var stream = File.Create(MODEL_NAME))
{
    context.Model.ConvertToOnnx(model, data, stream);
}

## Fill in Input Data

![ONNX Netron Image](onnx-netron.PNG)

### That's all folks!